In [ ]:
import sqlite3
# Connect to the database
conn = sqlite3.connect('./data/playwright_script.db')
cursor = conn.cursor()

res = cursor.execute("SELECT * FROM tests")
items = res.fetchall()

print( "there are {} data.".format(len(items)))

In [ ]:
from pathlib import Path
import json
import os

def db_query(sql_query, params=None):
    conn = sqlite3.connect('./data/playwright_script.db')
    cursor = conn.cursor()
    if params:
        cursor.execute(sql_query, params)
    else:
        cursor.execute(sql_query)
    results = cursor.fetchall()
    conn.commit()
    conn.close()
    return results

In [ ]:
def extract_html_information(html_path):
    # naive solution

    #TODO: Changeback to 'ISO-8859-1'
    with open(html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    return html_content[:20]

In [ ]:
def path_fix(path):
    return os.path.normpath(path).replace('\\', '/')
    
def extract_extra_operations(script1_path, script2_path):
    # read scripts
    #TODO: Changeback to 'ISO-8859-1'
    with open(script1_path, 'r', encoding='utf-8') as f1:
        script1_lines = f1.readlines()

    #TODO: Changeback to 'ISO-8859-1'
    with open(script2_path, 'r', encoding='utf-8') as f2:
        script2_lines = f2.readlines()

    script1_lines = [line.strip() for line in script1_lines if line.strip()]
    script2_lines = [line.strip() for line in script2_lines if line.strip()]

    # local extrat part
    extra_operations = '\n'.join([line for line in script2_lines if line not in script1_lines])

    return extra_operations

In [ ]:
def prompt_engineering(steps, html):
    # TODO: Added context_prompt and uncommented prompt
    context_prompt = (
        "<image>\n"
        "Task:\n"
        "You are a test automation script writer. I will describe a UI test in German and you will generate "
        "Playwright test code for the given webpage. Strictly follow these instructions:\n"
        "1. Don't explain the code, just generate the code block itself.\n"
        "2. You get some HTML elements and its attributes from the website. You can use playwright locators to find "
        "the elements by their attributes.\n"
        "3. Use the precondition code to set up the initial state. You must continue the code.\n"
        "4. Follow the steps in the ui test description to perform actions on the website.\n"
    )
    prompt = context_prompt
    
    # naive solution
    #prompt = f'Write type script code using the playwrite library to test the components discribed in the use case. \n USE-CASE: {steps}'
    
    return prompt

In [ ]:
def convert_db_item_to_json_dict(uid, idn, prefix='./'):
    # placeholder
    steps = None
    script = None
    screenshot_pre = None
    html_pre = None
    script_pre = None
    
    # get item with id
    x, y = idn.split('.')
    items = db_query(f"SELECT * FROM tests WHERE ID==?", (f'{x}.{y}',))
    if len(items) == 0:
        print("Error: target test case not found!")
        return -1
    item = items[0]
    steps = item[1]
    script = path_fix(prefix + item[5])
    
    # load precondition information
    if y == '1':
        # it's first step in the uitest
        screenshot_pre = '0_1.png'#prefix + 'screenshot\\0_1.png'
        html_pre = path_fix(prefix + 'html\\0_1.html')
        script_pre = path_fix(prefix + 'test_script\\0_1.spec.ts')
    else:
        items_pre = db_query(f"SELECT * FROM tests WHERE ID==?", (f'{x}.{str(int(y)-1)}',))
        if len(items_pre) == 0:
            print("Error: precondition item not found!")
            return -1
        item_pre = items_pre[0]
        html_pre = path_fix(prefix + item_pre[3])
        screenshot_pre = path_fix(prefix + path_fix(item_pre[4])).split('/')[-1]
        script_pre = path_fix(prefix + item_pre[5])
    
    # extract information from html
    html_pre = extract_html_information(html_pre)
    
    # prompt engineering
    inp = prompt_engineering(steps, html_pre)

    # compare the scripts
    # TODO: I uncommented this code line to retrieve the full code not just a single line
    #script = extract_extra_operations(script_pre, script)
    with open(script, 'r', encoding='utf-8') as f2:
        script = f2.read()
    
    # generate json dict
    json_data = {
        # TODO: Change from uid to str(uid)
                "id": str(uid),
                "image": screenshot_pre,
                "conversations": [
                    {
                        "from": "human",
                        "value": inp
                    },
                    {
                        "from": "gpt",
                        "value": script
                    }
                ]
            }
    
    return json_data

In [ ]:
from tqdm import tqdm

# extract data from db and convert to json file

# TODO: Change back to ids = [i[0] for i in db_query(f"SELECT * FROM tests")]
ids = list(filter(lambda x: not x.endswith(".1"), [i[0] for i in db_query(f"SELECT * FROM tests")]))

prefix = './data/'
json_output_path_train = './data/finetune/dataset_train.json'
json_output_path_val = './data/finetune/dataset_val.json'
json_data_list_train = []
json_data_list_val = []

val_perc = 0.2

for uid, idn in tqdm(enumerate(ids)):
    if uid > round(len(tqdm(enumerate(ids)))*1-val_perc):
        json_data_val = convert_db_item_to_json_dict(uid, idn, prefix)
        json_data_list_val.append(json_data_val)
        
    else:
        json_data_train = convert_db_item_to_json_dict(uid, idn, prefix)
        json_data_list_train.append(json_data_train)


        
with open(path_fix(json_output_path_val), 'w') as json_file:
    json.dump(json_data_list_train, json_file, indent=4)

with open(path_fix(json_output_path_train), 'w') as json_file:
    json.dump(json_data_list_train, json_file, indent=4)


# train val test split


In [142]:
import numpy as np
import os, os.path

In [266]:
np.random.seed(17)

perc_val = 0.2
perc_test = 0.2

In [263]:
dir = '../notebooks/test_script'
steps = (len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir,name)) and name != '.gitkeep'])) 
print(steps)

100


In [492]:
dir_list = os.listdir(dir)
dir_list.remove('.gitkeep')
tc_list = [i for i in range(1,31)]

np.random.seed(17)
idx = np.random.permutation(tc_list)
print(idx, len(idx))

test_ids_ctr = round(len(idx)* perc_test)
val_ids_ctr = round((len(idx) - test_ids_ctr) * perc_val)
train_ids_ctr = len(idx) - val_ids_ctr - test_ids_ctr

print(train_ids_ctr, val_ids_ctr, test_ids_ctr)
ids_train = idx[0:train_ids_ctr]
ids_val = idx[train_ids_ctr:train_ids_ctr+val_ids_ctr]
ids_test = idx[train_ids_ctr+val_ids_ctr:]
print(ids_train,"Length: ", len(ids_train))
print(ids_val,"Length: ", len(ids_val))
print(ids_test,"Length: ", len(ids_test))

#ctrl_ids = np.concatenate((ids_train, ids_val, ids_test))
#print(ctrl_ids.sort() == idx.sort())
#print(idx)

[ 1 19 22 12  6 24 28 20 25 27  4  3 11  9 10 29 15  2 21 13  8  5 26 14
 17 23  7 30 18 16] 30
19 5 6
[ 1 19 22 12  6 24 28 20 25 27  4  3 11  9 10 29 15  2 21] Length:  19
[13  8  5 26 14] Length:  5
[17 23  7 30 18 16] Length:  6


In [493]:
def triple_split(perc_val = 0.2, perc_test = 0.2, seed=17, dir='../notebooks/test_script'):
    
    steps = (len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir,name)) and name != '.gitkeep'])) 
    # Lists for all test cases and all test steps
    ts_list = os.listdir(dir)
    ts_list.remove('.gitkeep')
    tc_list = [i for i in range(1,31)]
    
    np.random.seed(seed)
    idx = np.random.permutation(tc_list)
    
    test_ids_ctr = round(len(idx)* perc_test)
    val_ids_ctr = round((len(idx) - test_ids_ctr) * perc_val)
    train_ids_ctr = len(idx) - val_ids_ctr - test_ids_ctr
    logger.debug(train_ids_ctr, val_ids_ctr, test_ids_ctr)
    
    ids_train = idx[0:train_ids_ctr]
    ids_val = idx[train_ids_ctr:train_ids_ctr+val_ids_ctr]
    ids_test = idx[train_ids_ctr+val_ids_ctr:]
    logger.debug(ids_train,"Length: ", len(ids_train))
    logger.debug(ids_val,"Length: ", len(ids_val))
    logger.debug(ids_test,"Length: ", len(ids_test))
    
    return ids_train, ids_val, ids_test, ts_list, steps
    

In [494]:
def balance_tests(ids_train, ids_test, ids_val, perc_test, perc_val, ts_list, steps):

    trigger_flag = False
    
    count_tr = 0
    for element in ts_list: 
        el = element[0:2].strip("_") 
        if int(el) in ids_train:
            count_tr += 1     
    est_ctr_tr = round(steps * (1 - perc_test) * (1 - perc_val))

    count_v = 0
    for element in ts_list: 
        el = element[0:2].strip("_") 
        if int(el) in ids_val:
            count_v += 1
    est_ctr_v = round(steps * (1 - perc_test) * perc_val)
    
    count_te = 0
    for element in ts_list: 
        el = element[0:2].strip("_") 
        if int(el) in ids_test:
            count_te += 1     
    est_ctr_te = round(steps * perc_test)   
    
    logger.debug(count_tr, "VERSUS", est_ctr_tr)
    logger.debug(count_v, "VERSUS", est_ctr_v)
    logger.debug(count_te, "VERSUS", est_ctr_te)
     
    if count_te <= est_ctr_te - 3:
        if count_tr >= est_ctr_tr + 3:
            ids_test = np.concatenate((ids_train[-1:], ids_test[:]))
            ids_train = ids_train[:-1]
            trigger_flag = True
        elif count_v >= est_ctr_v + 3:
            ids_test = np.concatenate((ids_val[-1:], ids_test[:] ))
            ids_val = ids_val[:-1]
            trigger_flag = True
    
    if count_v <= est_ctr_v - 3:
        if count_te >= est_ctr_te + 3:
            ids_val = np.concatenate((ids_test[-1:], ids_val[:]))
            ids_test = ids_test[:-1]
            trigger_flag = True
        if count_tr >= est_ctr_tr + 3:
            ids_val = np.concatenate((ids_train[-1:], ids_val[:]))
            ids_train = ids_train[:-1]
            trigger_flag = True
            
    if count_tr <= est_ctr_tr - 3:
        if count_te >= est_ctr_te + 3:
            ids_train = np.concatenate((ids_test[-1:], ids_train[:]))
            ids_test = ids_test[:-1]
            trigger_flag = True
        elif count_v >= est_ctr_v + 3:
            ids_train = np.concatenate((ids_val[-1:], ids_train[:]))
            ids_val = ids_val[:-1]
            trigger_flag = True
            
    logger.debug(trigger_flag)
    return ids_train, ids_test, ids_val, trigger_flag       

In [491]:
import numpy as np
import os, os.path
import time

from src.utils.logger import setup_logger
logger = setup_logger(__name__, level='INFO')

def train_val_test_split(perc_val = 0.2, perc_test = 0.2, balanced = False):
    
    ids_train, ids_val, ids_test, ts_list, steps = triple_split(perc_val = 0.2, perc_test = 0.2, seed=17, dir='../notebooks/test_script')
    
    trigger_flag = True
    dl_timer = time.process_time() # Sollte eig. nicht vorkommen können, dient der Absicherung
    
    while balanced and trigger_flag and (time.process_time() - dl_timer) < 5:
        ids_train, ids_test, ids_val, trigger_flag = balance_tests(ids_train, ids_test, ids_val, perc_test, perc_val, ts_list, steps)
    if (time.process_time() - dl_timer) >= 5:
        logger.error("Broke Deadlock constraint in train_val_test_split")
    return ids_train, ids_val, ids_test

# Abstellgleis

In [ ]:
count_compl = 0
for element in dir_list: 
    el = element[0:2].strip("_") 
    if int(el) in idx:
        count_compl += 1
    else: 
        print("missing nr", el)
print(count_compl)

count_tr = 0
for element in dir_list: 
    el = element[0:2].strip("_") 
    if int(el) in ids_train:
        count_tr += 1     
est_ctr_tr = round(steps *(1 - perc_test) * (1 - perc_test))
print(count_tr, "VERSUS", est_ctr_tr)

count_v = 0
for element in dir_list: 
    el = element[0:2].strip("_") 
    if int(el) in ids_val:
        count_v += 1
est_ctr_v = round(steps *(1 - perc_test) * perc_test)
print(count_v, "VERSUS", est_ctr_v)

count_te = 0
for element in dir_list: 
    el = element[0:2].strip("_") 
    if int(el) in ids_test:
        count_te += 1     
est_ctr_te = round(steps* perc_test) 
print(count_te, "VERSUS", est_ctr_te)

In [ ]:
l1 = [1,2,3 ]
l2 = [4,5,6 ]

l3 = np.concatenate((l2,l1))
l3